In [1]:
import numpy as np
from torch.utils.data.dataset import TensorDataset
import random
from sklearn.preprocessing import MinMaxScaler
from collections import deque
import torch

In [2]:
def GetAllFeatureLabel(filenames,label_start_idx):

    all_sample_feature = None
    all_sample_label = None
    for i in range(len(filenames)):
        data = np.loadtxt(filenames[i], delimiter=',', encoding='utf-8-sig')
        cur_feature = data[:, 0:label_start_idx]
        cur_label = np.full((len(cur_feature),), i)
        if all_sample_feature is None:
            all_sample_feature = cur_feature
            all_sample_label = cur_label
        else:
            all_sample_feature = np.concatenate((all_sample_feature, cur_feature), axis = 0 )
            all_sample_label = np.concatenate((all_sample_label, cur_label), axis = 0)
    return all_sample_feature, all_sample_label

In [3]:
def TLFeatureLabel(features, labels, TL):

    if len(labels) % TL == 0:
        a_feature_shape = features.shape[1]
        all_TL_feature = np.reshape(features,(-1,TL,a_feature_shape))
        all_TL_label = []
        for sample_idx in range(0,len(labels),TL):
            all_TL_label.append(labels[sample_idx])
        all_TL_label = np.array(all_TL_label)

    else:
        all_TL_feature = []
        all_TL_label = []
        for sample_idx in range(0, len(features), TL):
            cur_sample_feature = []
            cur_sample_label = labels[sample_idx]
            for k in range(TL):
                cur_sample_feature.append(features[sample_idx+k])
            cur_sample_feature = np.array(cur_sample_feature)
            all_TL_feature.append(cur_sample_feature)
            all_TL_label.append(cur_sample_label)
        all_TL_feature = np.array(all_TL_feature)
        all_TL_label = np.array(all_TL_label)
    return all_TL_feature, all_TL_label

In [4]:
def SplitPrivateOpen(features, labels, private_percent, open_percent, class_cat, open_fixed):
    if open_fixed == False:
        shuffle_in_unison(features, labels)
    feature_by_label = []
    for i in range(class_cat):
        feature_by_label.append(deque())
    for i in range(len(labels)):
        cur_label = labels[i]
        feature_by_label[cur_label].append(features[i])
    each_class_cnt = []
    for i in range(class_cat):
        each_class_cnt.append(len(feature_by_label[i]))

    private_set_feature = []
    private_set_label = []
    open_set_feature = []
    open_set_label = []
    for i in range(class_cat):
        private_cnt = int(each_class_cnt[i] * private_percent)
        open_cnt = int(each_class_cnt[i] * open_percent)
        cur_cnt = 0
        while cur_cnt < private_cnt and len(feature_by_label[i]) > 0:
            feature = feature_by_label[i].popleft()
            feature = np.array(feature)
            private_set_feature.append(feature)
            private_set_label.append(i)
            cur_cnt += 1
        cur_cnt = 0
        while cur_cnt < open_cnt and len(feature_by_label[i]) > 0:
            feature = feature_by_label[i].popleft()
            feature = np.array(feature)
            # print(feature.dtype)
            open_set_feature.append(feature)
            open_set_label.append(i)
            cur_cnt += 1
    return private_set_feature, private_set_label, open_set_feature, open_set_label

In [5]:
def FlatTLAugData(feature,each_line_shape):
    target_reshape = (-1,) + (each_line_shape,)
    return np.reshape(feature, target_reshape)

In [6]:
def MinMaxOpenPrivate(private_feature,open_feature):
    scaler = MinMaxScaler()
    scaler.fit(open_feature)
    open_feature = scaler.transform(open_feature)
    private_feature = scaler.transform(private_feature)
    return private_feature, open_feature, scaler


In [7]:
def TLAggData(feature,each_line_shape,TL):
    feature = np.reshape(feature,(-1,TL,each_line_shape))
    return feature

def SplitPrivate(feature,label,client_cnt,class_cat,iid,average):
    feature = np.array(feature)
    label = np.array(label)
    if (iid == True):
        shuffle_in_unison(feature, label)
    else:
         feature, label = shuffle_non_iid( feature, label,client_cnt, class_cat, average)
    private_length = len(label)

    start_idx = 0
    split_cnt = private_length // client_cnt
    private_datasets = []
    end_idx = start_idx + split_cnt
    for i in range(client_cnt):
        sub_feature = feature[start_idx:end_idx]
        sub_label = label[start_idx:end_idx]
        sub_feature = np.array(sub_feature)
        sub_label = np.array(sub_label)

        train_dataset = GetDataset(sub_feature, sub_label)
        private_datasets.append(train_dataset)

        start_idx = end_idx
        end_idx += split_cnt
    return private_datasets

In [8]:
def DilSplitPrivate(feature, label, client_cnt, class_cat, alpha, seed):
    alpha = [alpha]*(client_cnt*class_cat)
    probality = np.random.dirichlet(alpha,1).transpose()
    sum_pro = 0
    for i in range(len(probality)):
        sum_pro += probality[i]
    probality = np.reshape(probality,(client_cnt,class_cat))
    total_cnt = len(feature)

    que_features, que_labels = Queued(feature, label, class_cat)
    each_label_total_cnt = []
    for i in range(len(que_features)):
        each_label_total_cnt.append(len(que_features[i]))
    private_datasets = []
    for client_id in range(client_cnt):
        cur_client_feature = []
        cur_client_label = []
        cur_client_probality = probality[client_id]
        for label_id in range(class_cat):
            cur_label_cnt = int( total_cnt * cur_client_probality[label_id])
            if cur_label_cnt == 0 and label_id + 1 > len(cur_client_probality):
                cur_client_probality[label_id + 1] += cur_client_probality[label_id]
            added_cnt = 0
            while added_cnt < cur_label_cnt and len(que_features[label_id]) > 0:
                cur_client_feature.append(que_features[label_id].popleft())
                cur_client_label.append(que_labels[label_id].popleft())
                added_cnt += 1
        cur_client_feature = np.array(cur_client_feature)
        cur_client_label = np.array(cur_client_label)
        train_dataset = GetDataset(cur_client_feature, cur_client_label)
        private_datasets.append(train_dataset)
    return  private_datasets

In [9]:
def Queued(features, labels, class_cat):
    sorted_feature = []
    sorted_label = []
    sorted_dict = {}
    for i in range(len(features)):
        cur_label = labels[i]
        if cur_label in sorted_dict:
            sorted_dict[cur_label].append(features[i])
        else:
            sorted_dict[cur_label] = []
            sorted_dict[cur_label].append(features[i])
    queued_feature = []
    queued_label = []
    for i in range(class_cat):
        if i in sorted_dict:
            cur_feature = sorted_dict[i]
            cur_label = [i] * (len(cur_feature))
            cur_feature = deque(cur_feature)
            cur_label = deque(cur_label)
            queued_feature.append(cur_feature)
            queued_label.append(cur_label)
    return queued_feature, queued_label

In [10]:
def BinarySplitPrivate(feature,label,client_cnt,class_cat = 2,iid = False):
    features = [None] * class_cat
    labels = [None] * class_cat
    each_label_nums = [0] * class_cat
    each_client_samples_cnt = len(label) / client_cnt
    for i in range(len(label)):
        cur_label = label[i]
        cur_feature = feature[i]
        if labels[cur_label] is not None:
            labels[cur_label].append(cur_label)
            features[cur_label].append(cur_feature)
        else:
            labels[cur_label] = [cur_label]
            features[cur_label] = [cur_feature]
        each_label_nums[cur_label] += 1
    start_idx = [0] * class_cat
    all_datasets = []
    for i in range(client_cnt):
        cur_label = random.randint(0,class_cat-1)
        feature_more,label_more = get_feature_label_in_binary(cur_label,each_client_samples_cnt,0.9,features,labels,start_idx)
        feature_less,label_less = get_feature_label_in_binary((cur_label+1)%2,each_client_samples_cnt,0.1,features,labels,start_idx)
        if feature_more is not None and feature_less is not None:
            cur_client_feature = np.concatenate((feature_more,feature_less),axis = 0)
            cur_client_label = np.concatenate((label_more,label_less),axis = 0)
        elif feature_more is None:
            cur_client_feature = feature_less
            cur_client_label = label_less
        else:
            cur_client_feature = feature_more
            cur_client_label = label_more
        train_dataset = GetDataset(cur_client_feature, cur_client_label)
        all_datasets.append(train_dataset)
    return all_datasets

In [11]:
def get_feature_label_in_binary(cur_label,each_client_samples_cnt,ratio,features,labels,start_idx):
    cur_label_num = int(each_client_samples_cnt * ratio)
    if start_idx[cur_label] + cur_label_num < len(features[cur_label]):
        cur_client_feature = features[cur_label][start_idx[cur_label]:start_idx[cur_label]+cur_label_num]
        cur_client_label = labels[cur_label][start_idx[cur_label]:start_idx[cur_label]+cur_label_num]
    elif start_idx[cur_label] < len(features[cur_label]):
        cur_client_feature = features[cur_label][start_idx[cur_label]:]
        cur_client_label = labels[cur_label][start_idx[cur_label]:]
    else:
        cur_client_feature = None
        cur_client_label = None

    start_idx[cur_label] += cur_label_num
    return cur_client_feature, cur_client_label


In [12]:
def MakeDataset(features, labels, TL, need_min_max = True, need_TL = True, need_Transpose = True):
    if need_min_max :
        features = minMaxTrans(features)
    all_TL_feature = []
    all_TL_label = []

    for sample_idx in range(0, len(features), TL):
        cur_sample_feature = []
        cur_sample_label = labels[sample_idx]

        if need_TL == True:
            for k in range(TL):
                cur_sample_feature.append(features[sample_idx+k])
        else:
            cur_sample_feature = features[sample_idx]
        cur_sample_feature = np.array(cur_sample_feature)

        if need_Transpose:
            cur_sample_feature = np.transpose(cur_sample_feature, (1,0))
        all_TL_feature.append(cur_sample_feature)
        all_TL_label.append(cur_sample_label)
    all_TL_feature = np.array(all_TL_feature)
    all_TL_label = np.array(all_TL_label)
    shuffle_in_unison(all_TL_feature, all_TL_label)
    all_TL_feature, all_TL_label = torch.FloatTensor(all_TL_feature), torch.LongTensor(all_TL_label)
    dataset = torch.utils.data.TensorDataset(all_TL_feature, all_TL_label)
    return dataset

In [13]:
def SplitByLabel(dataset, private_percent, open_percent, class_cat):
    cat_feature = []
    for i in range(class_cat):
        cat_feature.append(deque())
    for fea, label in dataset:
        cat_feature[label].append(fea)
    cat_cnt = []
    for i in range(class_cat):
        cat_cnt.append(len(cat_feature[i]))
    private_set_feature = []
    private_set_label = []
    open_set_feature = []
    open_set_label = []
    for i in range(class_cat):
        private_cnt = int(cat_cnt[i] * private_percent)
        open_cnt = int(cat_cnt[i] * open_percent)
        cur_cnt = 0
        while cur_cnt < private_cnt and len(cat_feature[i]) > 0:
            feature = cat_feature[i].popleft()
            feature = np.array(feature)
            private_set_feature.append(feature)
            private_set_label.append(i)
            cur_cnt += 1

        cur_cnt = 0
        while cur_cnt < open_cnt and len(cat_feature[i]) > 0:
            feature = cat_feature[i].popleft()
            feature = np.array(feature)
            open_set_feature.append(feature)
            open_set_label.append(i)
            cur_cnt += 1
    return private_set_feature, private_set_label, open_set_feature, open_set_label

In [14]:
def SplitAsMultiDatasets(feature, label, iid, client_cnt, class_cat, average = False):
    print("Split As Datasets......")
    feature = np.array(feature)
    label = np.array(label)
    if (iid == True):
        shuffle_in_unison(feature, label)
    else:
         feature, label = shuffle_non_iid( feature, label,client_cnt, class_cat, average)
    private_length = len(label)

    start_idx = 0
    split_cnt = private_length // client_cnt

    end_idx = start_idx + split_cnt
    private_datasets = []
    for i in range(client_cnt):
        sub_feature = feature[start_idx:end_idx]
        sub_label = label[start_idx:end_idx]
        sub_feature = np.array(sub_feature)
        sub_label = np.array(sub_label)

        train_dataset = GetDataset(sub_feature, sub_label)
        private_datasets.append(train_dataset)

        start_idx = end_idx
        end_idx += split_cnt

    return private_datasets

In [15]:
def minMaxTrans(data):
    min_max = MinMaxScaler()
    data = min_max.fit_transform(data)
    return data

In [16]:
def shuffle_in_unison(a, b):

    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [17]:
def ShuffleDataset(dataset):

    feature, label = dataset[:]
    feature, label = np.array(feature), np.array(label)
    shuffle_in_unison(feature,label)
    feature,label = torch.tensor(feature),torch.tensor(label)
    dataset = TensorDataset(feature,label)
    return dataset

In [18]:
def shuffle_non_iid(feature, label, client_cnt, class_cat, average = False):

    feature, label = sorted_by_label(feature, label, class_cat)
    sample_shape = feature[0].shape
    taregt_reshape = (-1,) + sample_shape

    if average == False:
        parts = client_cnt
    else:
        parts = client_cnt * 2

    num_of_each_part = len(label) // parts
    feature_parts_list = []
    label_parts_list = []
    start_idx = 0
    end_idx = num_of_each_part
    for i in range(parts):
        part_feature = feature[start_idx:end_idx]
        part_label = label[start_idx:end_idx]
        feature_parts_list.append(part_feature)
        label_parts_list.append(part_label)
        start_idx = end_idx
        end_idx += num_of_each_part
    shuffle_in_unison(feature_parts_list, label_parts_list)
    part_feature = np.array(feature_parts_list)
    part_label = np.array(label_parts_list)
    part_feature = np.reshape(part_feature, taregt_reshape)
    part_label = part_label.reshape(-1)
    return part_feature, part_label

In [19]:
def sorted_by_label(feature, label, class_cat):
    sorted_feature = []
    sorted_label = []
    sorted_dict = {}
    for i in range(len(feature)):
        cur_label = label[i]
        if cur_label in sorted_dict:
            sorted_dict[cur_label].append(feature[i])
        else:
            sorted_dict[cur_label] = []
            sorted_dict[cur_label].append(feature[i])
    for i in range(class_cat):
        if i in sorted_dict:
            sorted_feature += sorted_dict[i]
            labels = [i] * len(sorted_dict[i])
            sorted_label += labels
    sorted_feature = np.array(sorted_feature)
    sorted_label = np.array(sorted_label)
    return sorted_feature, sorted_label

In [20]:
def GetLoader(data_feature, data_label, batch_size ):
    dataset = GetDataset(data_feature,data_label)
    data_loader = GetLoaderFromDataset(dataset,batch_size)
    return data_loader

def GetLoaderFromDataset(dataset, batch_size,shuffle = True):
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=False
    )
    return data_loader

def GetDataset(feature, label):
    feature, label = torch.FloatTensor(feature), torch.LongTensor(label)
    dataset = torch.utils.data.TensorDataset(feature, label)
    return dataset

In [21]:
def TLAggragate(features, TL):
    all_TL_feature = []
    for sample_idx in range(0, len(features), TL):
        cur_sample_feature = []
        for k in range(TL):
            cur_sample_feature.append(features[sample_idx+k])
        cur_sample_feature = np.array(cur_sample_feature)
        all_TL_feature.append(cur_sample_feature)
    all_TL_feature = np.array(all_TL_feature)
    random.shuffle(all_TL_feature)
    return all_TL_feature

def GetFeatureFromOpenDataset(open_dataset, start_idx, end_idx):
    feature, label = open_dataset[start_idx:end_idx]
    return feature, label

def SplitDataFrame(data_frame, TL,percent1, percent2):
    data_array = np.array(data_frame)
    data_array = TLAggragate(data_array, TL)
    aug_sample_cnt = len(data_array)
    part1_cnt = int(percent1 * aug_sample_cnt)
    part2_cnt = int(percent2 * aug_sample_cnt)
    part1_array = data_array[0:part1_cnt]
    part2_array = data_array[part1_cnt:part1_cnt + part2_cnt]
    return part1_array, part2_array


In [22]:
import torch
import numpy as np

In [23]:
def TrainWithFeatureLabel(dev,feature,label,batchsize,model,opt,loss_func):
    dataset = torch.utils.data.TensorDataset(feature, label)
    avg_loss = TrainWithDataset(dev,dataset,batchsize,model,opt,loss_func)
    return avg_loss

def TrainWithDataset(dev,dataset,batchsize,model,opt,loss_func):
    data_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=batchsize,
                    shuffle=True,
                )
    model = model.to(dev)
    model.train()
    total_loss = 0
    for batch_idx, (batch_feature, batch_label) in enumerate(data_loader):
        opt.zero_grad()
        batch_feature = batch_feature.to(dev)
        batch_label = batch_label.to(dev)
        preds = model(batch_feature)
        loss = loss_func(preds, batch_label)
        loss.backward()
        opt.step()
        total_loss += loss.item() * batch_label.size(0)
    avg_loss = total_loss / len(dataset)
    return avg_loss

In [24]:
def EvalWithFeatureLabel(dev,feature,label,batchsize,model,loss_func):
    dataset = torch.utils.data.TensorDataset(feature, label)
    avg_loss = EvalWithDataset(dev,dataset,batchsize,model,loss_func)
    return avg_loss


def EvalWithDataset(dev,dataset,batchsize,model,loss_func):
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=batchsize, shuffle=True)
    total_loss = 0
    with torch.no_grad():
        for batch_idx, (feature, label) in enumerate(test_loader):
            feature = feature.to(dev)
            label = label.to(dev)
            out = model(feature)
            loss = loss_func(out, label)
            total_loss += (loss.item() * label.size(0))
    test_loss = total_loss / (len(dataset))
    return test_loss

In [25]:
def Predict(dev,feature,model,model_out_len):
    with torch.no_grad():
        model = model.to(dev)
        feature = feature.to(dev)
        logits = model(feature)
        pred_label = Logits2PredLabel(logits,model_out_len)
        return pred_label

def Logits2PredLabel(logits,model_out_len):
    "pred -> hard label"
    with torch.no_grad():
        if model_out_len == 1:
            prediction = torch.round(torch.sigmoid(logits))
        else:
            _,prediction = torch.max(logits,1)
    return prediction

def Predict2SoftLabel(dev,feature,model,model_out_len):
    with torch.no_grad():
        model = model.to(dev)
        feature = feature.to(dev)
        logits = model(feature)
        logits = Logits2Soft(logits,model_out_len)
        return logits

def Logits2Soft(logits,model_out_len):
    sigmoid = torch.nn.Sigmoid()
    softmax = torch.nn.Softmax(1)
    with torch.no_grad():
        if model_out_len == 1:
            logits = sigmoid(logits)
            soft_max_logits = torch.zeros(len(logits),2)
            for i in range(len(logits)):
                soft_max_logits[i] = torch.tensor([1-logits[i].item(),logits[i].item()])
            logits = soft_max_logits
        else:
            logits = softmax(logits)
        return logits

In [26]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def Metrics(true_label, pred_label):
    """
    Calculate and return performance metrics like Accuracy, F1 Score, Precision, and Recall.
    """
    # Convert tensors to numpy arrays for sklearn metrics
    true_label = true_label.cpu().numpy()
    pred_label = pred_label.cpu().numpy()

    # Calculate metrics
    accuracy = accuracy_score(true_label, pred_label)
    f1 = f1_score(true_label, pred_label, average='weighted')  # Use 'weighted' for multi-class
    precision = precision_score(true_label, pred_label, average='weighted')  # Use 'weighted' for multi-class
    recall = recall_score(true_label, pred_label, average='weighted')  # Use 'weighted' for multi-class

    return accuracy, f1, precision, recall

In [27]:
def PredictWithDisUnknown(dev, open_feature,
                        classify_model,classify_model_len_out_tensor,
                        discri_model, discri_model_len_out_tensor,
                        class_cat):

    discri_model = discri_model.to(dev)
    classify_model = classify_model.to(dev)
    average_tensor = [1.0 / class_cat ] * class_cat
    average_tensor = torch.tensor(average_tensor)

    with torch.no_grad():
        open_feature = open_feature.to(dev)
        wait_to_dis_label = classify_model(open_feature)
        wait_to_dis_label = Logits2Soft(wait_to_dis_label,classify_model_len_out_tensor)
        dis_label = discri_model(open_feature)
        dis_label = Logits2PredLabel(dis_label,discri_model_len_out_tensor)
        for i in range(len(dis_label)):
            if dis_label[i].item() == 1:
                wait_to_dis_label[i] = average_tensor.clone()
        return wait_to_dis_label

In [28]:
def PredictAvg(dev,dataset,bounds,model):
    print()
    print("pred avg")
    print(bounds)
    each_label_avg_logit = {}
    soft_max = torch.nn.Softmax(1)
    model = model.to(dev)
    for i in range(len(bounds)):
        cur_class_start_idx = bounds[i][0]
        cur_class_end_idx = bounds[i][1]
        cur_label = i
        if cur_class_start_idx == cur_class_end_idx:
            continue
        else:
            cur_class_feature, cur_class_label = dataset[cur_class_start_idx:cur_class_end_idx]
            with torch.no_grad():
                cur_class_feature = cur_class_feature.to(dev)
                pred_label = model(cur_class_feature)
                pred_label = soft_max(pred_label)
                pred_label = torch.mean(pred_label, dim=0)
            cur_label = cur_class_label[0].item()
            each_label_avg_logit[cur_label] = pred_label.detach().clone()
    print("end pred avg")
    return each_label_avg_logit

In [29]:
def PredictFilter(dev, open_feature, classify_model,classify_model_len_out_tensor, class_cat, theta):
    print()
    print("in predict filter")
    print("theta = {}".format(theta))
    classify_model = classify_model.to(dev)
    average_tensor = [1.0 / class_cat ] * class_cat
    average_tensor = torch.tensor(average_tensor)

    with torch.no_grad():
        open_feature = open_feature.to(dev)
        wait_to_dis_label = classify_model(open_feature)
        wait_to_dis_label = Logits2Soft(wait_to_dis_label,classify_model_len_out_tensor)
        if theta < 0:
            max_num, pred_label = torch.max(wait_to_dis_label,1)
            theta = max_num.median()
        for i in range(len(wait_to_dis_label)):
            pred_label, pred_pro = torch.argmax(wait_to_dis_label[i]), torch.max(wait_to_dis_label[i])
            if pred_pro < theta:
                wait_to_dis_label[i] = average_tensor.clone()
        return wait_to_dis_label

def HardLabel(soft_label):
    sample_cnt = len(soft_label)
    class_cat = len(soft_label[0])
    boundary = 1 / class_cat
    hard_label = [0] * sample_cnt
    for i in range(sample_cnt):
        cur_soft_label = soft_label[i]
        pred_label, pred_pro = torch.argmax(cur_soft_label), torch.max(cur_soft_label)
        hard_label[i] = pred_label.item() if pred_pro > boundary else class_cat
    return hard_label

In [30]:
def HardLabelVoteHard(all_client_hard_label, class_cat):
    client_cnt = len(all_client_hard_label)
    sample_cnt = len(all_client_hard_label[0])
    pred_labels = []
    label_votes_none_cnt = 0
    for i in range(sample_cnt):
        label_votes = [0] * class_cat
        for j in range(client_cnt):
            cur_client_cur_sample_hard_label = all_client_hard_label[j][i]
            pred_label = cur_client_cur_sample_hard_label
            if pred_label != class_cat:
                label_votes[pred_label] += 1
        if (len(label_votes) == 0):
            label_votes_none_cnt += 1
        max_vote_nums = max(label_votes)
        max_vote_idx = label_votes.index(max_vote_nums)
        pred_labels.append(max_vote_idx)
    pred_labels = torch.tensor(pred_labels)
    return pred_labels

def HardLabelVoteOneHot(all_client_hard_label, class_cat):
    client_cnt = len(all_client_hard_label)
    sample_cnt = len(all_client_hard_label[0])
    pred_labels = []
    all_vote_tensor = []
    label_votes_none_cnt = 0
    for i in range(sample_cnt):
        label_votes = [0] * class_cat
        for j in range(client_cnt):
            cur_client_cur_sample_hard_label = all_client_hard_label[j][i]
            pred_label = cur_client_cur_sample_hard_label
            if pred_label != class_cat:
                label_votes[pred_label] += 1
        if (len(label_votes) == 0):
            label_votes_none_cnt += 1
        max_vote_nums = max(label_votes)
        max_vote_idx = label_votes.index(max_vote_nums)
        pred_labels.append(max_vote_idx)
    all_one_hot_tensor = []
    for i in range(len(pred_labels)):
        cur_label = [0.0] * class_cat
        cur_label[pred_labels[i]] = 1.0
        all_one_hot_tensor.append(cur_label)
    all_vote_tensor = torch.tensor(all_one_hot_tensor)
    print("len of pred = {}".format(len(pred_labels)))
    print()
    return all_vote_tensor

In [31]:
def OneHot2Label(one_hot_vectors):
    _,labels = torch.max(one_hot_vectors,1)
    labels = labels.double()
    return labels

def GetDeviceClientCnt(device_name, client_cnt, classify_model_out_len):
    if classify_model_out_len == 1:
        return 4
    else:
        if ((device_name == "Ennio_Doorbell/" or device_name == "Samsung_SNH_1011_N_Webcam/")):
            return int(client_cnt / 2) + 1

        else:
            return client_cnt

def GetDeviceClassCat(device_name,classify_model_out_len):
    if classify_model_out_len == 1:
        return 2
    if ((device_name == "Ennio_Doorbell/" or device_name == "Samsung_SNH_1011_N_Webcam/")):
        return 6
    else :
        return 11

In [32]:
def reshape_sample(feature):
    feature = np.reshape(feature, (-1, 23, 5))
    return feature

def PredUnknown(dev, feature, model, theta, model_out_len):
    sure_unknown = []
    wait_to_dis = []
    soft_max = torch.nn.Softmax(1)
    sigmoid = torch.nn.Sigmoid()
    model = model.to(dev)
    feature = feature.to(dev)
    with torch.no_grad():
        out = model(feature)
        if model_out_len == 1:
            out = sigmoid(out)
            soft_max_out = torch.zeros(len(out),2)
            for i in range(len(out)):
                soft_max_out[i] = torch.tensor([1-out[i].item(),out[i].item()])
            out = soft_max_out
        else:
            out = soft_max(out)
        max_num, pred_label = torch.max(out, 1)
        if theta < 0 :
            theta = max_num.median()
        for i in range(len(max_num)):
            if max_num[i] < theta:
                sure_unknown.append(feature[i])
            else:
                wait_to_dis.append(feature[i])
    if len(sure_unknown) == 0:
        return None
    sure_unknown = torch.stack(sure_unknown)

    return sure_unknown

def LabelFeature(feature,label):
    labels = [label] * len(feature)
    labels = torch.tensor(labels)
    return feature, labels

In [33]:
def DisUnknown(dev, client, dis_rounds, batchsize, dis_train_feature,theta):
    dis_train_feature = dis_train_feature.detach().clone()
    sure_unknown_feature = PredUnknown(dev, dis_train_feature, client.classify_model, theta,
                                       client.classify_model_out_len)
    if sure_unknown_feature is None:
        return False

    unknown_label_num = -1
    known_label_num = -1
    if client.discri_model_out_len == 1:
        unknown_label_num = 1.0
        known_label_num = 0.0
    else:
        unknown_label_num = 1
        known_label_num = 0

    sure_unknown_feature,sure_unknown_label = LabelFeature(sure_unknown_feature,unknown_label_num)
    sure_known_feature, _ = client.classify_dataset[:]
    sure_known_feature = sure_known_feature.detach().clone()

    sure_known_feature,sure_known_label = LabelFeature(sure_known_feature, known_label_num)

    sure_unknown_feature = sure_unknown_feature.to(dev)
    sure_known_feature = sure_known_feature.to(dev)

    dis_feature = torch.cat((sure_known_feature,sure_unknown_feature),0)

    sure_known_label = sure_known_label.to(dev)
    sure_unknown_label = sure_unknown_label.to(dev)

    dis_label = torch.cat((sure_known_label,sure_unknown_label),0)
    cpu_dev = torch.device("cpu")
    dis_feature = dis_feature.to(cpu_dev)
    dis_label = dis_label.to(cpu_dev)
    dis_dataset = torch.utils.data.TensorDataset(dis_feature, dis_label)
    dis_dataset = ShuffleDataset(dis_dataset)

    for r in range(dis_rounds):
        TrainWithDataset(dev, dis_dataset, batchsize, client.discri_model, client.discri_opt, client.discri_loss_func)

    return True

In [34]:
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [35]:
class SSFL_IDS_Client():
    def __init__(self, idx,
                classify_dataset:torch.utils.data.Dataset, classify_model:nn.Module,classify_model_out_len,
                classify_lr:float,
                discri_model,discri_model_out_len,discri_lr):

        self.classify_model = classify_model
        self.classify_dataset = classify_dataset
        self.class_cat = classify_model_out_len if classify_model_out_len > 1 else 2
        self.each_class_cnt = [0] * self.class_cat
        for _,label in self.classify_dataset:
            self.each_class_cnt[label.item()] += 1
        self.classify_lr = classify_lr
        self.c_idx = idx
        self.classify_opt = optim.Adam(self.classify_model.parameters(), lr=self.classify_lr)
        self.discri_model = discri_model
        self.discri_lr = discri_lr
        self.discri_opt = optim.Adam(self.discri_model.parameters(), lr=self.discri_lr)
        self.discri_model_out_len = discri_model_out_len
        if discri_model_out_len == 1:
            self.discri_loss_func = nn.BCEWithLogitsLoss()
        else:
            self.discri_loss_func = nn.CrossEntropyLoss()
        self.classify_model_out_len = classify_model_out_len
        if classify_model_out_len == 1:
            self.hard_label_loss_func = nn.BCEWithLogitsLoss()
            self.feature, self.label = self.classify_dataset[:]
            self.label = self.label.double()
            self.classify_dataset = torch.utils.data.TensorDataset(self.feature,self.label)
        else:
            self.hard_label_loss_func = nn.CrossEntropyLoss()
        self.soft_label_loss_func = SSFL_IDS_CELoss()

In [36]:
class SSFL_IDS_CELoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pred_pro, target_tensor):
        pred_pro = F.log_softmax(pred_pro, dim=1)
        out = -1 * pred_pro * target_tensor
        return out.sum() / len(pred_pro)

In [37]:
class SSFL_IDS_Server():
    def __init__(self, model,model_out_len, clients, dist_lr):
        self.model = model
        self.clients = clients
        self.client_cnt = len(clients)
        self.model_out_len = model_out_len
        self.dist_lr = dist_lr
        self.dist_opt = optim.Adam(self.model.parameters(), lr=self.dist_lr)
        self.soft_label_loss_func = SSFL_IDS_CELoss()
        if model_out_len != 1:
            self.hard_label_loss_func = nn.CrossEntropyLoss()
        else:
            self.hard_label_loss_func = nn.BCEWithLogitsLoss()

In [38]:
def Create_SSFL_IDS_Client(client_idx, private_dataset ,classify_model,classify_model_out_len, lr, discri_model,discri_model_out_len,discri_lr):
    client = SSFL_IDS_Client(client_idx,private_dataset,classify_model,classify_model_out_len,lr,discri_model,discri_model_out_len,discri_lr)
    return client

def Create_SSFL_IDS_Server(server_model,classify_model_out_len,clients,dist_lr):
    server = SSFL_IDS_Server(server_model,classify_model_out_len,clients,dist_lr)
    return server

def CreateDataset(configs, dataset_name = "NBaIoT"):
    if dataset_name == "NBaIoT":
        return create_NBaIoT(configs)

In [39]:
# Updated dataset path for Google Colab
def create_NBaIoT(configs):
    prefix = "/kaggle/input/nba-iot/nba_iot_1000/"

    device_names = [
        "Danmini_Doorbell/", "Ecobee_Thermostat/", "Philips_B120N10_Baby_Monitor/",
        "Provision_PT_737E_Security_Camera/", "Provision_PT_838_Security_Camera/", "SimpleHome_XCS7_1002_WHT_Security_Camera/",
        "SimpleHome_XCS7_1003_WHT_Security_Camera/", "Ennio_Doorbell/", "Samsung_SNH_1011_N_Webcam/",
    ]
    attack_names = [
        "benign", "g_combo", "g_junk", "g_scan", "g_tcp", "g_udp",
        "m_ack", "m_scan", "m_syn", "m_udp", "m_udpplain"
    ]
    if configs["classify_model_out_len"] == 1:
        attack_names = ["benign", "attack"]

    all_device_train_feature, all_device_train_label = None, None
    all_device_open_feature, all_device_open_label = None, None
    all_device_private_feature, all_device_private_label = [], []
    all_device_test_feature, all_device_test_label = None, None
    device_cnt = len(device_names)

    if not configs["load_data_from_pickle"]:
        for d_idx in range(device_cnt):
            cur_device_class_cat = GetDeviceClassCat(device_names[d_idx], configs["classify_model_out_len"])
            train_filenames, test_filenames = [], []

            for i in range(len(attack_names)):
                if i < cur_device_class_cat:
                    train_filename = prefix + device_names[d_idx] + attack_names[i] + "_train.csv"
                    test_filename = prefix + device_names[d_idx] + attack_names[i] + "_test.csv"
                    train_filenames.append(train_filename)
                    test_filenames.append(test_filename)

            train_feature, train_label = GetAllFeatureLabel(train_filenames, configs["label_start_idx"])
            private_feature, private_label, open_feature, open_label = SplitPrivateOpen(
                train_feature, train_label, configs["private_percent"], configs["open_percent"],
                cur_device_class_cat, False
            )
            all_device_private_feature.append(private_feature)
            all_device_private_label.append(private_label)

            if all_device_open_feature is None:
                all_device_open_feature, all_device_open_label = open_feature, open_label
            else:
                all_device_open_feature = np.concatenate((all_device_open_feature, open_feature), axis=0)
                all_device_open_label = np.concatenate((all_device_open_label, open_label), axis=0)

            if all_device_train_feature is None:
                all_device_train_feature, all_device_train_label = train_feature, train_label

            test_feature, test_label = GetAllFeatureLabel(test_filenames, configs["label_start_idx"])
            if all_device_test_feature is None:
                all_device_test_feature, all_device_test_label = test_feature, test_label
            else:
                all_device_test_feature = np.concatenate((all_device_test_feature, test_feature), axis=0)
                all_device_test_label = np.concatenate((all_device_test_label, test_label), axis=0)

    scaler = MinMaxScaler()
    scaler.fit(all_device_open_feature)
    all_device_open_feature = scaler.transform(all_device_open_feature)
    all_device_open_feature = reshape_sample(all_device_open_feature)
    open_dataset = GetDataset(all_device_open_feature, all_device_open_label)
    open_dataset = ShuffleDataset(open_dataset)

    all_device_test_feature = scaler.transform(all_device_test_feature)
    all_device_test_feature = reshape_sample(all_device_test_feature)
    test_dataset = GetDataset(all_device_test_feature, all_device_test_label)

    private_datasets = []
    for d_idx in range(device_cnt):
        cur_device_class_cat = GetDeviceClassCat(device_names[d_idx], configs["classify_model_out_len"])
        cur_device_client_cnt = GetDeviceClientCnt(device_names[d_idx], configs["device_client_cnt"],
                                                   configs["classify_model_out_len"])
        cur_device_private_feature = all_device_private_feature[d_idx]
        cur_device_private_label = all_device_private_label[d_idx]
        cur_device_private_feature = scaler.transform(cur_device_private_feature)
        cur_device_private_feature = reshape_sample(cur_device_private_feature)

        if configs["iid"]:
            cur_device_private_datasets = SplitPrivate(
                cur_device_private_feature, cur_device_private_label,
                cur_device_client_cnt, cur_device_class_cat, configs["iid"], configs["data_average"]
            )
        elif configs["split"] == "dile":
            cur_device_private_datasets = DilSplitPrivate(
                cur_device_private_feature, cur_device_private_label,
                cur_device_client_cnt, cur_device_class_cat,
                configs["alpha_of_dile"], configs["seed"]
            )
        else:  # "equally"
            cur_device_private_datasets = SplitPrivate(
                cur_device_private_feature, cur_device_private_label,
                cur_device_client_cnt, cur_device_class_cat, configs["iid"], configs["data_average"]
            )
        private_datasets.append(cur_device_private_datasets)

    return test_dataset, private_datasets, open_dataset

In [40]:
import torch.nn as nn
import torch.nn.functional as F

In [41]:
class CNN(nn.Module):
    def __init__(self, output_len):
        super().__init__()

        self.conv1 = nn.Conv1d(23, 64, 3, padding=1)

        self.conv2 = nn.Conv1d(64, 64, 3, padding = 1)

        self.conv3 = nn.Conv1d(64, 64, 3, padding=1)
        self.dropoutcv2 = nn.Dropout(p=0.3)

        self.conv4 = nn.Conv1d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv1d(64, 128, 3, padding=1)

        self.conv6 = nn.Conv1d(128, 128, 3, stride=1, padding=1)
        self.conv7 = nn.Conv1d(128, 128, 3, stride=2, padding=1)
        self.dropoutcv3 = nn.Dropout(p=0.3)
        self.conv8 = nn.Conv1d(128, 128, 3, stride=2, padding=1)
        self.dropoutcv4 = nn.Dropout(p=0.3)

        self.fc1 = nn.Linear(256,128)

        self.fc2 = nn.Linear(128, output_len)
        self.output_cnt = output_len

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropoutcv2(x)

        x = self.conv4(x)
        x = self.conv5(x)

        x = self.conv6(x)
        x = F.relu(x)
        x = self.conv7(x)
        x = F.relu(x)

        x = self.dropoutcv3(x)
        x = self.conv8(x)
        x = F.relu(x)
        x = self.dropoutcv4(x)

        x = x.view(x.size()[0], -1)

        x = self.fc1(x)

        x = self.fc2(x)
        if self.output_cnt == 1:
            x = x.squeeze(dim=-1)
        return x

def GetNbaIotModel(output_len):
    model = CNN(output_len)
    return model

In [42]:
import sys
import torch
import numpy as np

In [43]:
def SSFL_IDS(conf, dev, clients, server, test_dataset, open_dataset):
    comm_cnt = conf["comm_cnt"]
    open_idx_set_cnt = conf["open_idx_set_cnt"]
    batchsize = conf["batchsize"]
    train_rounds = conf["train_rounds"]
    dis_rounds = conf["discri_rounds"]
    dist_rounds = conf["dist_rounds"]
    theta = conf["theta"]
    labels = conf["labels"]
    first_train_rounds = conf["first_train_rounds"]
    class_cat = conf["classify_model_out_len"] if conf["classify_model_out_len"] > 1 else 2
    dis_train_cnt = 10000
    start_idx = 0
    end_idx = start_idx + open_idx_set_cnt
    open_len = len(open_dataset)

    # List to store metrics for each round
    all_metrics = []

    for e in range(comm_cnt):
        sure_unknown_none = set()
        all_client_hard_label = []
        open_feature, open_label = GetFeatureFromOpenDataset(open_dataset, start_idx, end_idx)
        if open_idx_set_cnt > open_len:
            global_logits = torch.zeros(open_len, len(labels))
        else:
            global_logits = torch.zeros(open_idx_set_cnt, len(labels))
        client_cnt = len(clients)
        participate = 0

        print("Round {} Stage I".format(e+1))

        for c_idx in range(client_cnt):

            print("Client {} Training...".format(c_idx+1))

            cur_client = clients[c_idx]
            cur_train_rounds = train_rounds if e != 0 else first_train_rounds

            if len(cur_client.classify_dataset) == 0:
                continue

            for train_r in range(cur_train_rounds):
                TrainWithDataset(dev, cur_client.classify_dataset, batchsize, cur_client.classify_model,
                                              cur_client.classify_opt, cur_client.hard_label_loss_func)

            if sum(i > 0 for i in cur_client.each_class_cnt) == 1:
                continue
            else:
                participate += 1
            dis_train_feature, _ = GetFeatureFromOpenDataset(open_dataset, 0, dis_train_cnt)

            succ = DisUnknown(dev, cur_client, dis_rounds, batchsize, dis_train_feature, theta)

            if succ == False:
                sure_unknown_none.add(c_idx)

            cur_client_open_feature = open_feature.detach().clone()

            if c_idx not in sure_unknown_none:
                local_logit = PredictWithDisUnknown(dev, cur_client_open_feature,
                                                    cur_client.classify_model, cur_client.classify_model_out_len,
                                                    cur_client.discri_model, cur_client.discri_model_out_len,
                                                    len(labels))
                copy_local_logit = local_logit.detach().clone()

                hard_label = HardLabel(copy_local_logit)
                all_client_hard_label.append(hard_label)

        print()

        global_logits = HardLabelVoteHard(all_client_hard_label, class_cat)

        print("Round {} Stage II".format(e+1))

        for c_idx in range(len(clients)):
            cur_client = clients[c_idx]
            print("Client {} Distillation Training...".format(c_idx+1))

            for r in range(dist_rounds):
                cur_global_logits = global_logits.detach().clone()
                cur_client_open_feature = open_feature.detach().clone()
                if cur_client.classify_model_out_len != 1:
                    TrainWithFeatureLabel(dev, cur_client_open_feature, cur_global_logits, batchsize,
                                                       cur_client.classify_model, cur_client.classify_opt,
                                                       cur_client.hard_label_loss_func)
                else:
                    cur_global_logits = OneHot2Label(cur_global_logits)
                    TrainWithFeatureLabel(dev, cur_client_open_feature, cur_global_logits, batchsize,
                                                       cur_client.classify_model, cur_client.classify_opt,
                                                       cur_client.hard_label_loss_func)
        print()

        print("Server Training...")

        for dist_i in range(dist_rounds):
            cur_global_logits = global_logits.detach().clone()
            server_open_feature = open_feature.detach().clone()
            if server.model_out_len != 1:
                TrainWithFeatureLabel(dev, server_open_feature, cur_global_logits, batchsize,
                                                          server.model, server.dist_opt, server.hard_label_loss_func)
            else:
                cur_global_logits = OneHot2Label(cur_global_logits)
                TrainWithFeatureLabel(dev, server_open_feature, cur_global_logits, batchsize,
                                                          server.model, server.dist_opt, server.hard_label_loss_func)

        test_feature, test_label = test_dataset[:]
        pred_label = Predict(dev, test_feature, server.model, server.model_out_len)
        
        # Calculate and store metrics for this round
        accuracy, f1, precision, recall = Metrics(test_label, pred_label)
        all_metrics.append({
            "round": e + 1,
            "accuracy": accuracy,
            "f1_score": f1,
            "precision": precision,
            "recall": recall
        })

        print(f"Round {e+1} Test Metrics:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print()

    # Print final performance metrics
    print("Final Performance Metrics:")
    final_metrics = all_metrics[-1]  # Get metrics from the last round
    print(f"Accuracy: {final_metrics['accuracy']:.4f}")
    print(f"F1 Score: {final_metrics['f1_score']:.4f}")
    print(f"Precision: {final_metrics['precision']:.4f}")
    print(f"Recall: {final_metrics['recall']:.4f}")
      

    # Return the metrics for further analysis
    return all_metrics

def SSFL_IDS_NBaIoT():
    configs = {
        "comm_cnt": 30,
        "device_client_cnt": 11,
        "private_percent": 0.9,
        "batchsize": 100,
        "iid": False,
        "need_dist": True,
        "open_percent": 0.1,
        "label_lr": 0.00005,
        "dist_lr": 0.00005,
        "discri_lr": 0.00005,
        "train_rounds": 5,
        "discri_rounds": 3,
        "dist_rounds": 15,
        "first_train_rounds": 5,
        "open_idx_set_cnt": 10000,
        "discri_cnt": 10000,
        "dist_T": 0.1,
        "need_SA": False,
        "test_batch_size": 256,
        "label_start_idx": 115,
        "test_round": 1,
        "data_average": True,
        "labels": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "clien_need_dist_opt": False,
        "discri_model_out_len": 1,
        "classify_model_out_len": 11,
        "sample_cnt": 1000,
        "random": True,
        "vote": True,
        "seed": 7,
        "load_data_from_pickle": False,
        "soft_label": False,
        "num_after_float": 4,
        "theta": -1,
        "split": "dile",
        "alpha_of_dile": 0.1,
    }

    if configs["seed"] is not None:
        np.random.seed(configs["seed"])

    device_names = [
        "Danmini_Doorbell/", "Ecobee_Thermostat/", "Philips_B120N10_Baby_Monitor/",
        "Provision_PT_737E_Security_Camera/", "Provision_PT_838_Security_Camera/", "SimpleHome_XCS7_1002_WHT_Security_Camera/",
        "SimpleHome_XCS7_1003_WHT_Security_Camera/","Ennio_Doorbell/", "Samsung_SNH_1011_N_Webcam/",
    ]

    device_cnt = len(device_names)

    clients = []
    dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    client_idx = 0
    test_dataset, private_dataset, open_dataset = CreateDataset(configs, "NBaIoT")

    for d_idx in range(device_cnt):
        cur_device_client_cnt = GetDeviceClientCnt(device_names[d_idx], configs["device_client_cnt"], configs["classify_model_out_len"])
        cur_device_private_datasets = private_dataset[d_idx]
        for i in range(cur_device_client_cnt):
            classify_model_out_len = configs["classify_model_out_len"]
            classify_model = GetNbaIotModel(classify_model_out_len)
            discri_model_out_len = configs["discri_model_out_len"]
            discri_model = GetNbaIotModel(discri_model_out_len)
            client = Create_SSFL_IDS_Client(client_idx, cur_device_private_datasets[i], classify_model, classify_model_out_len,
                                            configs["label_lr"], discri_model, discri_model_out_len, configs["discri_lr"])
            clients.append(client)
            client_idx += 1

    server_model = GetNbaIotModel(configs["classify_model_out_len"])
    server = Create_SSFL_IDS_Server(server_model, configs["classify_model_out_len"], clients, configs["dist_lr"])
    all_metrics = SSFL_IDS(configs, dev, clients, server, test_dataset, open_dataset)

if __name__ == "__main__":
    SSFL_IDS_NBaIoT()

Round 1 Stage I
Client 1 Training...
Client 2 Training...
Client 3 Training...
Client 4 Training...
Client 5 Training...
Client 6 Training...
Client 7 Training...
Client 8 Training...
Client 9 Training...
Client 10 Training...
Client 11 Training...
Client 12 Training...
Client 13 Training...
Client 14 Training...
Client 15 Training...
Client 16 Training...
Client 17 Training...
Client 18 Training...
Client 19 Training...
Client 20 Training...
Client 21 Training...
Client 22 Training...
Client 23 Training...
Client 24 Training...
Client 25 Training...
Client 26 Training...
Client 27 Training...
Client 28 Training...
Client 29 Training...
Client 30 Training...
Client 31 Training...
Client 32 Training...
Client 33 Training...
Client 34 Training...
Client 35 Training...
Client 36 Training...
Client 37 Training...
Client 38 Training...
Client 39 Training...
Client 40 Training...
Client 41 Training...
Client 42 Training...
Client 43 Training...
Client 44 Training...
Client 45 Training...
Cli

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Round 1 Test Metrics:
Accuracy: 0.1011
F1 Score: 0.0186
Precision: 0.0102
Recall: 0.1011

Round 2 Stage I
Client 1 Training...
Client 2 Training...
Client 3 Training...
Client 4 Training...
Client 5 Training...
Client 6 Training...
Client 7 Training...
Client 8 Training...
Client 9 Training...
Client 10 Training...
Client 11 Training...
Client 12 Training...
Client 13 Training...
Client 14 Training...
Client 15 Training...
Client 16 Training...
Client 17 Training...
Client 18 Training...
Client 19 Training...
Client 20 Training...
Client 21 Training...
Client 22 Training...
Client 23 Training...
Client 24 Training...
Client 25 Training...
Client 26 Training...
Client 27 Training...
Client 28 Training...
Client 29 Training...
Client 30 Training...
Client 31 Training...
Client 32 Training...
Client 33 Training...
Client 34 Training...
Client 35 Training...
Client 36 Training...
Client 37 Training...
Client 38 Training...
Client 39 Training...
Client 40 Training...
Client 41 Training...
C

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Round 2 Test Metrics:
Accuracy: 0.1471
F1 Score: 0.0626
Precision: 0.0831
Recall: 0.1471

Round 3 Stage I
Client 1 Training...
Client 2 Training...
Client 3 Training...
Client 4 Training...
Client 5 Training...
Client 6 Training...
Client 7 Training...
Client 8 Training...
Client 9 Training...
Client 10 Training...
Client 11 Training...
Client 12 Training...
Client 13 Training...
Client 14 Training...
Client 15 Training...
Client 16 Training...
Client 17 Training...
Client 18 Training...
Client 19 Training...
Client 20 Training...
Client 21 Training...
Client 22 Training...
Client 23 Training...
Client 24 Training...
Client 25 Training...
Client 26 Training...
Client 27 Training...
Client 28 Training...
Client 29 Training...
Client 30 Training...
Client 31 Training...
Client 32 Training...
Client 33 Training...
Client 34 Training...
Client 35 Training...
Client 36 Training...
Client 37 Training...
Client 38 Training...
Client 39 Training...
Client 40 Training...
Client 41 Training...
C

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Round 28 Test Metrics:
Accuracy: 0.7403
F1 Score: 0.7054
Precision: 0.7210
Recall: 0.7403

Round 29 Stage I
Client 1 Training...
Client 2 Training...
Client 3 Training...
Client 4 Training...
Client 5 Training...
Client 6 Training...
Client 7 Training...
Client 8 Training...
Client 9 Training...
Client 10 Training...
Client 11 Training...
Client 12 Training...
Client 13 Training...
Client 14 Training...
Client 15 Training...
Client 16 Training...
Client 17 Training...
Client 18 Training...
Client 19 Training...
Client 20 Training...
Client 21 Training...
Client 22 Training...
Client 23 Training...
Client 24 Training...
Client 25 Training...
Client 26 Training...
Client 27 Training...
Client 28 Training...
Client 29 Training...
Client 30 Training...
Client 31 Training...
Client 32 Training...
Client 33 Training...
Client 34 Training...
Client 35 Training...
Client 36 Training...
Client 37 Training...
Client 38 Training...
Client 39 Training...
Client 40 Training...
Client 41 Training...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Round 29 Test Metrics:
Accuracy: 0.7437
F1 Score: 0.7095
Precision: 0.7205
Recall: 0.7437

Round 30 Stage I
Client 1 Training...
Client 2 Training...
Client 3 Training...
Client 4 Training...
Client 5 Training...
Client 6 Training...
Client 7 Training...
Client 8 Training...
Client 9 Training...
Client 10 Training...
Client 11 Training...
Client 12 Training...
Client 13 Training...
Client 14 Training...
Client 15 Training...
Client 16 Training...
Client 17 Training...
Client 18 Training...
Client 19 Training...
Client 20 Training...
Client 21 Training...
Client 22 Training...
Client 23 Training...
Client 24 Training...
Client 25 Training...
Client 26 Training...
Client 27 Training...
Client 28 Training...
Client 29 Training...
Client 30 Training...
Client 31 Training...
Client 32 Training...
Client 33 Training...
Client 34 Training...
Client 35 Training...
Client 36 Training...
Client 37 Training...
Client 38 Training...
Client 39 Training...
Client 40 Training...
Client 41 Training...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
